In [58]:
import pandas as pd
import pickle
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer


In [2]:
motor=pd.read_csv('motor_final.csv')
non_motor=pd.read_csv('non_motor_final.csv')

In [3]:
merged = motor.merge(non_motor, how='outer', on=['PATNO'])

In [14]:
merged.columns 

Index(['PATNO', 'NP1RTOT', 'NP1PTOT', 'NP2PTOT', 'PDTRTMNT', 'NP3TOT',
       'DYSKPRES', 'NHY', 'NP4TOT', 'MSEADLG', 'ENROLL_AGE_x', 'SDMTOTAL',
       'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT', 'REMSLEEP_TOT', 'QUIP_A',
       'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E', 'MoCA_score', 'LNS_TOT',
       'HVLT_TOT_Recall', 'HVLT_DCR_REC', 'HVLT_RETENTION', 'GDS_TOT',
       'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy', 'BJLOT_TOT', 'COHORT'],
      dtype='object')

In [13]:
merged.drop('COHORT_x',axis=1,inplace=True)

In [15]:
merged.to_csv('merged.csv')

In [17]:
merged.isnull().sum()

PATNO                0
NP1RTOT            384
NP1PTOT            384
NP2PTOT            384
PDTRTMNT           384
NP3TOT             384
DYSKPRES           384
NHY                384
NP4TOT             384
MSEADLG            384
ENROLL_AGE_x       384
SDMTOTAL             2
STAI_TOT             2
SFT_TOT              2
SCOPA_AUT_TOT        2
REMSLEEP_TOT         2
QUIP_A               2
QUIP_B               2
QUIP_C               2
QUIP_D               2
QUIP_E               2
MoCA_score           2
LNS_TOT              2
HVLT_TOT_Recall      2
HVLT_DCR_REC         2
HVLT_RETENTION       2
GDS_TOT              2
GDS_Depressed        2
ESS_TOT              2
ESS_Sleepy           2
BJLOT_TOT            2
COHORT               2
dtype: int64

In [18]:
for cohort_value in merged['COHORT'].unique():
    # Filter the DataFrame based on cohort value
    cohort_df = merged[merged['COHORT'] == cohort_value]
    
    # Save the filtered DataFrame to a new CSV file
    cohort_df.to_csv(f'Merged_COHORT_{cohort_value}_data.csv',index=False)

In [59]:
df1 = pd.read_csv('Merged_COHORT_0.0_data.csv')
df2 = pd.read_csv('Merged_COHORT_1.0_data.csv')
df3 = pd.read_csv('Merged_COHORT_2.0_data.csv')
df4 = pd.read_csv('Merged_COHORT_3.0_data.csv')

In [60]:
# Define a function to fill missing values and apply KNN imputation
def fill_missing_and_impute(df):
    # Group DataFrame by 'PATNO' and fill missing values with median within each group
    grouped_df = df.groupby('PATNO').transform(lambda x: x.fillna(x.median()))

    # Add 'PATNO' column back to the filled DataFrame
    df_filled = pd.concat([df['PATNO'], grouped_df], axis=1)

    # Calculate the mean of each feature after filling missing values
    grouped_mean_df = df_filled.groupby('PATNO').mean().reset_index()

    # Apply KNN imputation to handle remaining missing values
    knn = KNNImputer()
    grouped_mean_df = pd.DataFrame(knn.fit_transform(grouped_mean_df), columns=grouped_mean_df.columns)
    
    return grouped_mean_df

# Apply the function to all four dataframes
df1_filled = fill_missing_and_impute(df1)
df2_filled = fill_missing_and_impute(df2)
df3_filled = fill_missing_and_impute(df3)
df4_filled = fill_missing_and_impute(df4)

In [61]:
f = pd.concat([df1_filled
               ,df2_filled,df3_filled,df4_filled])

In [62]:
f.sample(10)

,PATNO,NP1RTOT,NP1PTOT,NP2PTOT,PDTRTMNT,NP3TOT,DYSKPRES,NHY,NP4TOT,MSEADLG,...,LNS_TOT,HVLT_TOT_Recall,HVLT_DCR_REC,HVLT_RETENTION,GDS_TOT,GDS_Depressed,ESS_TOT,ESS_Sleepy,BJLOT_TOT,COHORT
848,101557.0,2.800000,5.500000,8.100000,0.800000,25.700000,0.0,2.000000,1.111111,92.000000,...,10.0,22.0,11.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0
945,113355.0,1.666667,6.166667,6.833333,0.666667,26.750000,0.0,2.000000,0.000000,97.500000,...,12.0,25.0,9.0,1.0,2.0,0.0,12.0,1.0,14.0,0.0
84,3350.0,0.153846,3.153846,0.307692,0.000000,2.923077,0.0,0.153846,0.000000,100.000000,...,14.0,32.0,11.0,0.0,0.0,0.0,6.0,0.0,14.0,1.0
32,3115.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,99.246154,...,12.0,19.0,11.0,1.0,2.0,0.0,10.0,0.0,14.0,1.0
1137,183263.0,0.000000,4.000000,0.000000,0.000000,1.500000,0.0,0.000000,0.000000,100.000000,...,12.0,33.0,12.0,1.0,0.0,0.0,3.0,0.0,11.0,3.0
939,112712.0,1.666667,5.666667,7.333333,0.666667,22.750000,0.0,1.750000,1.000000,93.571429,...,11.0,24.0,10.0,0.0,1.0,0.0,8.0,0.0,8.0,0.0
798,100898.0,0.000000,4.100000,7.000000,0.888889,35.571429,0.0,1.777778,0.444444,92.000000,...,13.0,21.0,9.0,0.0,0.0,0.0,5.0,0.0,14.0,0.0
1127,182518.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,...,10.0,27.0,9.0,0.0,1.0,0.0,9.0,0.0,13.0,3.0
693,106519.0,0.000000,4.000000,1.000000,0.000000,3.000000,0.0,0.000000,0.000000,100.000000,...,10.0,24.0,12.0,1.0,1.0,0.0,5.0,0.0,11.0,3.0
228,52414.0,0.500000,3.250000,0.125000,0.000000,0.750000,0.0,0.000000,0.000000,100.000000,...,11.0,29.0,11.0,1.0,1.0,0.0,1.0,0.0,14.0,3.0


In [30]:
f.isnull().sum()

PATNO              0
NP1RTOT            0
NP1PTOT            0
NP2PTOT            0
PDTRTMNT           0
NP3TOT             0
DYSKPRES           0
NHY                0
NP4TOT             0
MSEADLG            0
ENROLL_AGE_x       0
SDMTOTAL           0
STAI_TOT           0
SFT_TOT            0
SCOPA_AUT_TOT      0
REMSLEEP_TOT       0
QUIP_A             0
QUIP_B             0
QUIP_C             0
QUIP_D             0
QUIP_E             0
MoCA_score         0
LNS_TOT            0
HVLT_TOT_Recall    0
HVLT_DCR_REC       0
HVLT_RETENTION     0
GDS_TOT            0
GDS_Depressed      0
ESS_TOT            0
ESS_Sleepy         0
BJLOT_TOT          0
COHORT             0
dtype: int64

In [63]:
f['COHORT'].value_counts()

COHORT
0.0    1752
3.0    1737
1.0     357
2.0      81
Name: count, dtype: int64

In [33]:
X =  f.drop(['COHORT'],axis=1)
y= f['COHORT']

In [64]:
f.to_csv('merged_mnm.csv')

In [36]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [37]:
svm = SVC(kernel="rbf", gamma="auto",probability=True)
svm.fit(X_train,y_train)
prediction = svm.predict(X_test)

accuracy_SVM = accuracy_score(prediction, y_test)
print(accuracy_SVM)

0.6292857142857143


In [38]:
dec = DecisionTreeClassifier(random_state=4)
dec.fit(X_train,y_train)
pred = dec.predict(X_test)
accuracy_DEC = accuracy_score(pred, y_test)
print(accuracy_DEC)

0.9371428571428572


In [44]:
rand = RandomForestClassifier(max_depth=25, random_state=0)
rand.fit(X_train,y_train)
rand_pred = rand.predict(X_test)
rand_accuracy = accuracy_score(rand_pred,y_test)
print(rand_accuracy)

0.9607142857142857


In [40]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
cross_val_results = cross_val_score(rand, X, y, cv=kf)
print(f'Cross-Validation Results (Accuracy): {cross_val_results}')
print(f'Mean Accuracy: {cross_val_results.mean()}')

Cross-Validation Results (Accuracy): [0.94020356 0.93384224 0.91592357 0.92484076 0.94649682]
Mean Accuracy: 0.9322613896047066


In [41]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, and y_test are already defined

# Initialize and train the gradient boosting classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)

# Make predictions on the test set
gb_pred = gb_clf.predict(X_test)

# Calculate accuracy
gb_accuracy = accuracy_score(gb_pred, y_test)
print("Gradient Boosting Classifier Accuracy:", gb_accuracy)

Gradient Boosting Classifier Accuracy: 0.9535714285714286


In [53]:
from sklearn.ensemble import VotingClassifier
# Create a Soft Voting Classifier
labels = ['Decision ', 'Random Forest','GB']
voting_clf_hard = VotingClassifier(
	estimators=[
        
		(labels[1], rand), # Include the second classifier (Random Forest)
		(labels[2],gb_clf ), # Include the third classifier (Naive Bayes)
	],
	voting='soft' # Specify soft voting, where class probabilities are combined
)

In [54]:
voting_clf_hard.fit(X_train,y_train)
V_soft_pred = voting_clf_hard.predict(X_test)
voting_soft_accuracy = accuracy_score(V_soft_pred,y_test)
print(voting_soft_accuracy)

0.9535714285714286


In [56]:
import pickle
import joblib
model=rand

with open("model_Merged.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("model_Merged.pkl", "rb") as f:
    model = pickle.load(f)
    
joblib.dump(model, "model_Merged.joblib")
model = joblib.load("model_Merged.joblib")